import the necessary

In [2]:
import numpy as np
import pandas as pd
from keras.layers import LSTM
from keras.layers import Dense
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns

read the data using pandas read_csv

In [3]:
oil = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv')
sample = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv')
holiday = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv')
stores = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/stores.csv')
train_data = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv')
test_data = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/test.csv')
transactions = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv')

merge all data into one dataframe with date as the index

In [4]:
train_oil = pd.merge(train_data, oil, on = 'date', how = 'left')
train_oil = train_oil.fillna(method = 'ffill')
test_oil = pd.merge(test_data,oil, on = 'date', how = 'left')
test_oil = test_oil.fillna(method = 'ffill')
train_holiday = pd.merge(train_oil,holiday, on = 'date', how = 'left')
train_holiday = train_holiday.fillna('Empty')
test_holiday = pd.merge(test_oil, holiday, on = 'date', how = 'left')
test_holiday = test_holiday.fillna('Empty')
transactions = transactions.fillna(0)
train_transaction = pd.merge(train_holiday, transactions, on = ['date','store_nbr'], how = 'left')
test_transaction = pd.merge(test_holiday, transactions, on = ['date', 'store_nbr'], how = 'left')
train_store = pd.merge(train_transaction, stores , on = 'store_nbr', how = 'left')
test_store = pd.merge(test_transaction, stores, on = 'store_nbr', how = 'left')
transactions = transactions.fillna(0)

convert data into datetime format and seperate day month and year

In [5]:
train_store.date = pd.to_datetime(train_store.date)
train_store['day'] = train_store['date'].dt.day
train_store['month'] = train_store['date'].dt.month
train_store['year'] = train_store['date'].dt.year

In [6]:
test_store.date = pd.to_datetime(test_store.date)
test_store['day'] = test_store['date'].dt.day
test_store['month'] = test_store['date'].dt.month
test_store['year'] = test_store['date'].dt.year

drop the redundant column

In [7]:
train_stores = train_store.drop(columns = ['id','date','transactions','dcoilwtico','transferred', 'description', 'state','store_nbr'])
test_stores = test_store.drop(columns =  ['id','date','transactions','dcoilwtico','transferred', 'description', 'state','store_nbr'])

In [8]:
train_stores.head()

In [9]:
train_stores.info()

In [10]:
df = train_stores
ds = test_stores

In [11]:
df.rename(columns = {'type_x':'holiday', 'type_y':'shop_type'}, inplace = True)
ds.rename(columns = {'type_x':'holiday', 'type_y':'shop_type'}, inplace = True)

In [12]:
s = (df.dtypes == 'object')
object_cols = list(s[s].index)
print(object_cols)

splitting data into train and test

In [13]:
from sklearn.model_selection import train_test_split
X = df
X_test = ds
X.dropna(axis=0, subset=['sales'], inplace=True)
y = X.sales
X.drop(columns = ['sales'], axis = 1, inplace = True)
X_train, X_valid, y_train, y_valid = train_test_split(X, y,
                                                      train_size=0.8, test_size=0.2,
                                                      random_state=0)

converting categorical data to numerical data with one hot encoding and normalization of numerical data

In [14]:
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))
OH_cols_test = pd.DataFrame(OH_encoder.transform(X_test[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index
OH_cols_test.index = X_test.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)
num_X_test = X_test.drop(object_cols,axis = 1)

#normalization of numerical data (train, test and validation data)
train_df = num_X_train
train_df_mean = train_df.mean()
train_df_std = train_df.std()
train_df_norm = (train_df - train_df_mean)/train_df_std

valid_df = num_X_valid
valid_df_mean = valid_df.mean()
valid_df_std  = valid_df.std()
valid_df_norm = (valid_df - valid_df_mean)/valid_df_std

test_df = num_X_test
test_df_mean = test_df.mean()
test_df_std  = test_df.std()
test_df_norm = (test_df - test_df_mean)/test_df_std

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([train_df_norm, OH_cols_train], axis=1)
OH_X_valid = pd.concat([valid_df_norm, OH_cols_valid], axis=1)
OH_X_test = pd.concat([test_df_norm, OH_cols_test],axis = 1)

In [15]:
OH_X_train

In [16]:
y_train

In [17]:
# train_dates = pd.to_datetime(df['date'])

Random forest regression

In [18]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_absolute_error
# from sklearn import metrics

# model = RandomForestRegressor(n_estimators = 100, random_state = 1)
# model.fit(OH_X_train,y_train)
# prediction = model.predict(OH_X_valid)
# # print(metrics.accuracy_score(prediction,y_valid))
# print('mean absolute error :' , mean_absolute_error(y_valid,prediction))

LinearRegression

In [19]:
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import mean_absolute_error
# from sklearn import metrics
# lin_model = LinearRegression()
# lin_model.fit(OH_X_train,y_train)
# prediction = lin_model.predict(OH_X_valid)
# print('accuracy:',metrics.accuracy_score(prediction,y_valid))
# print('mean absolute error: ',mean_absolute_error(y_valid,prediction))

XG boost regressor

In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt
import xgboost as xgb
XG = xgb.XGBRegressor(objective = 'reg:squarederror' , learning_rate = 0.1,
                max_depth = 50, n_estimators = 100).fit(OH_X_train, y_train)
y_pred_XG = XG.predict(OH_X_valid)
def relu(x):
    relu = []
    for i in x: 
        if i < 0:
            relu.append(0)
        else:
            relu.append(i)
    return relu

# plt.scatter(y_valid, relu(y_pred_XG))
# plt.plot([10000*x for x in range(10)], [10000*x for x in range(10)], color = 'r')
# plt.xlabel("Reality")
# plt.ylabel("Predicted")
# plt.title('CatBoostRegressor')
# plt.show()
# plt.clf()
print(XG.score(OH_X_valid, y_valid))

save the predicted value in the csv file

In [ ]:
sub = XG.predict(OH_X_test)
sample['sales'] = relu(sub)
sample.to_csv('submission.csv', index=False)